<a href="https://colab.research.google.com/github/ankushkunayak/AnalysisProject/blob/main/Estimating_Healthcare_Insurance_Expenses_through_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#--- Import Pandas ---
import pandas as pd
#--- Read in dataset ----
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/insurance.csv')

df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
duplicates = df.duplicated().sum()
duplicates

1

In [ ]:
# Eliminating Duplications.
df.drop_duplicates(inplace = True)

df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:

null_values = df.isnull().sum()

#--- Inspect data ---
null_values

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder Instance
lab_encode = LabelEncoder()

# Label coding to 'sex' and 'smoker' cloumns
for col in ['sex', 'smoker']:
    df[col] = lab_encode.fit_transform(df[col])

#--- Inspect data ---
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:

one_hot_encode = pd.get_dummies(df['region'])

#--- Inspect data ---
one_hot_encode

,northeast,northwest,southeast,southwest
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
1333,0,1,0,0
1334,1,0,0,0
1335,0,0,1,0
1336,0,0,0,1


In [ ]:

df1 = pd.concat([df, one_hot_encode], axis=1)

df1.head()

,age,sex,bmi,children,smoker,region,charges,northeast,northwest,southeast,southwest
0,19,0,27.900,0,1,southwest,16884.92400,0,0,0,1
1,18,1,33.770,1,0,southeast,1725.55230,0,0,1,0
2,28,1,33.000,3,0,southeast,4449.46200,0,0,1,0
3,33,1,22.705,0,0,northwest,21984.47061,0,1,0,0
4,32,1,28.880,0,0,northwest,3866.85520,0,1,0,0


In [ ]:

df1.drop('region', axis=1, inplace=True)

df1.head()

,age,sex,bmi,children,smoker,charges,northeast,northwest,southeast,southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = ...
X = df1.drop('charges', axis=1)
y = df1["charges"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train
X_test
y_train
y_test

900      8688.85885
1064     5708.86700
1256    11436.73815
298     38746.35510
237      4463.20510
           ...     
534     13831.11520
542     13887.20400
760      3925.75820
1284    47403.88000
1285     8534.67180
Name: charges, Length: 268, dtype: float64

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
rand_forest_model = RandomForestRegressor(n_estimators=50, n_jobs=2, random_state=42)

neg_mse_scores = cross_val_score(rand_forest_model, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
performance = np.sqrt(-neg_mse_scores)
std = performance.std()

std

586.8826484248

In [ ]:

rand_forest_model.fit(X_train, y_train)
predictions = rand_forest_model.predict(X_test)
rounded_predictions = np.round(predictions, 2)
rounded_actual_charges = np.round(y_test.values, 2)
compare = pd.DataFrame({
    'Actual Charges': rounded_actual_charges[:10],
    'Predicted Charges': rounded_predictions[:10]
})

compare.head()

,Actual Charges,Predicted Charges
0,8688.86,10334.48
1,5708.87,6651.49
2,11436.74,12295.40
3,38746.36,42608.01
4,4463.21,6853.03
